In [1]:
import pandas as pd
import numpy as np
import sys, re, requests, collections, glob, time 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
import os
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
import json
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None
%matplotlib inline

In [45]:

seasonAverages = pd.read_csv('data/SeasonAverages.csv', header = 0, index_col = 0)

results_df = pd.read_csv("data/RegularSeasonCompactResults.csv", header = 0)
results_df = results_df[results_df["Season"] >= 2003][["WTeamID", "LTeamID", "Season"]]

print(results_df.head())


       WTeamID  LTeamID  Season
74048     1104     1328    2003
74049     1272     1393    2003
74050     1266     1437    2003
74051     1296     1457    2003
74052     1400     1208    2003
